In [1]:
import sys
sys.path.append('../src/models') 
sys.path.append('../src/data') 
import tensorflow as tf

In [2]:
from model1.dataloader import load_data
from model1.model_v0_3 import ContentBasedFiltering

In [3]:
train_dataset, cv_dataset, test_dataset, info = load_data(sample_train= 0.05, feature_info=True)

loading	 train_dataset
loaded:	 train_dataset
loading	 cv_dataset
loaded:	 cv_dataset
loading	 test_dataset
loaded:	 test_dataset
train_dataset: 5549 examples
cv_dataset: 12140 examples
test_dataset: 12747 examples


In [4]:
batch_size = 32

train_dataset = train_dataset.shuffle(buffer_size=len(train_dataset))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [5]:
cv_dataset = cv_dataset.batch(batch_size)

In [6]:
num_G_features = info['X_group_num_features']
num_T_features = info['X_technique_num_features']

In [7]:
model1 = ContentBasedFiltering (
    num_G_features=num_G_features, 
    num_T_features=num_T_features,
    Group_NN_width = 16,
    Group_NN_depth = 3,
    Technique_NN_width = 16,
    Technique_NN_depth = 3,)

In [8]:
model1.model.summary()

Model: "recsysNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_Group (InputLayer)       [(None, 463)]        0           []                               
                                                                                                  
 input_Technique (InputLayer)   [(None, 54)]         0           []                               
                                                                                                  
 Group_NN (Sequential)          (None, 32)           8512        ['input_Group[0][0]']            
                                                                                                  
 Technique_NN (Sequential)      (None, 32)           1968        ['input_Technique[0][0]']        
                                                                                     

In [9]:
epochs = 5
history = model1.train (
    train_dataset,
    val_data = cv_dataset,
    epochs= epochs
)

Epoch 1/5
174/174 [==============================] - 8s 6ms/step - loss: 0.5778 - val_loss: 0.6213
Epoch 2/5
174/174 [==============================] - 6s 6ms/step - loss: 0.5221 - val_loss: 0.5743
Epoch 3/5
174/174 [==============================] - 7s 6ms/step - loss: 0.4971 - val_loss: 0.6330
Epoch 4/5
174/174 [==============================] - 6s 6ms/step - loss: 0.5027 - val_loss: 0.5149
Epoch 5/5
174/174 [==============================] - 6s 6ms/step - loss: 0.4938 - val_loss: 0.5153


In [10]:
test_dataset = test_dataset.batch(batch_size)

loss, accuracy = model1.evaluate (test_dataset)

TypeError: cannot unpack non-iterable float object